# Creation of the CNN

### Importations 

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import pandas as pd
import time


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


In [2]:
print("Tmux a vraiment run cette schisse.")

Tmux a vraiment run cette schisse.


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [3]:
Dataset = pd.read_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")
Dataset.loc[0:3]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[3.395545610109257, 2.0550302704519794, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,21
1,0,"[3.2402107165724896, 1.8276250338752629, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,32
2,0,"[1.6801067878855174, 1.927483519251545, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,9
3,0,"[3.2065269137273935, 3.207686173511143, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,27


#### Defining our number of samples.

In [4]:
number_of_rooms = Dataset.shape[0]

### Loading the phase matrices in tensors with respect to the indexing shown in the dataframe.

In [269]:
#path containing the Training data on the machine you are working on.
#path = "/home/janjar/Dataset/Trainingset/"

data_matrix = np.zeros([number_of_rooms,4,129,390])
for i in range(number_of_rooms):
    fileName_matrix = Dataset.iloc[i]['Phase_Matrix']
    #print(fileName_matrix)
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(140, 4, 129, 390)

In [6]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,4,129,390)
data_matrix = data_matrix.type('torch.FloatTensor')
data_matrix.shape

torch.Size([140, 4, 129, 390])

### Function that reshape a label into the right format for the data_targets.

In [7]:
def prep_targets(index,label):
    target = torch.tensor(label)
    target = target.expand(390,1)
    return target

In [8]:
data_targets = torch.zeros([number_of_rooms], dtype=torch.float64)
for i in range(number_of_rooms):
    data_targets[i] = Dataset.iloc[i]['Label']
data_targets.shape   

torch.Size([140])

# Architecture 1.0 : vanilla CNN for DOA 

 In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the   sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Model

In [156]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 100):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, kernel_size=2)
        self.conv2 = nn.Conv2d(2, 4, kernel_size=2)
        self.conv3 = nn.Conv2d(4 ,8, kernel_size=2)
        self.fc1 = nn.Linear(390096, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden*4,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,37)
        
    
    def forward(self, x):
        x_0 = x[:,:,0,:,:].view(-1,1,129,390)
        x_1 = x[:,:,1,:,:].view(-1,1,129,390)
        x_2 = x[:,:,2,:,:].view(-1,1,129,390)
        x_3 = x[:,:,3,:,:].view(-1,1,129,390)

        x_0 = F.relu(self.conv1(x_0))         
        x_1 = F.relu(self.conv1(x_1))
        x_2 = F.relu(self.conv1(x_2))
        x_3 = F.relu(self.conv1(x_3))

        x_0 = F.relu(self.conv2(x_0))
        x_1 = F.relu(self.conv2(x_1))
        x_2 = F.relu(self.conv2(x_2))
        x_3 = F.relu(self.conv2(x_3))
        
        x_0 = F.relu(self.conv3(x_0))
        x_1 = F.relu(self.conv3(x_1))
        x_2 = F.relu(self.conv3(x_2))
        x_3 = F.relu(self.conv3(x_3)) 
        

        x_0 = F.relu(self.fc1(x_0.view(-1, 390096)))
        x_1 = F.relu(self.fc1(x_1.view(-1, 390096)))
        x_2 = F.relu(self.fc1(x_2.view(-1, 390096)))            
        x_3 = F.relu(self.fc1(x_3.view(-1, 390096))) 

        x = torch.cat((x_0,x_1,x_2,x_3),1)            

        x = F.softmax(self.fc2(x))

        x = F.softmax(self.fc3(x))

        return x            

In [243]:
class SimpleModel(nn.Module):

    def __init__(self, nb_hidden=50):
        super(SimpleModel, self).__init__()
        self.cl1 = nn.Conv2d(390, 8, kernel_size=2)
        self.cl2 = nn.Conv2d(8, 16, kernel_size=2)
        self.cl3 = nn.Conv2d(16, 32, kernel_size=2)
        self.fc1 = nn.Linear(4032, 200)
        self.fc2 = nn.Linear(200,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,37)
        #self.fc4 = nn.Linear(256,37)
 
 
    def forward(self, x):
       
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.cl3(x))
        #print(x.shape)
        x = F.relu(self.fc1(x.view(-1, 4032)))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))

        return x

    def compute_nb_errors(self,model, data_input, data_target, mini_batch_size):

        nb_data_errors = 0
        for b in range(0, data_input.size(0), mini_batch_size):
            a = model(data_input.narrow(0, b, mini_batch_size))
            val = torch.max(a,1)[1]
            for k in range(mini_batch_size):
                if data_target.data[b + k] != val[k]:
                    nb_data_errors = nb_data_errors + 1

        return nb_data_errors

### Preparing the Data

In [244]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,1,4,129,390)
    return new_train_input

In [245]:
def prep_input_model2(train_input):
    new_train_input = train_input.view(-1,1,390,4,129)
    return new_train_input

In [246]:
train_data = prep_input_model2(data_matrix)
train_data.shape

torch.Size([140, 1, 390, 4, 129])

In [247]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1)
    return new_train_input

In [248]:
train_targets = prep_labels_vanilla(data_targets)
train_targets = train_targets.to(dtype=torch.int64)
train_data, train_targets = Variable(train_data), Variable(train_targets)
train_data.shape

torch.Size([140, 1, 390, 4, 129])

### Spliting the Training Data into Training/Validation.

In [249]:
training_data = train_data[0:110]
training_targets = train_targets[0:110]
validation_data = train_data[110:140]
validation_targets = train_targets[110:140]
training_data.shape, validation_data.shape, training_targets.shape, validation_targets.shape,len(training_targets),len(validation_targets)


(torch.Size([110, 1, 390, 4, 129]),
 torch.Size([30, 1, 390, 4, 129]),
 torch.Size([110]),
 torch.Size([30]),
 110,
 30)

### Cutting the training data to test is the model runs

training_data = training_data[0:4]
training_targets = training_targets[0:4]
training_data[1]

### End of Sanity check.

## 1.2 Training the model

In [250]:
model = Net()

In [251]:
model2 = SimpleModel()

In [252]:
mini_batch_size = 5 
nb_epochs = 25
eta = 0.1 #learning rate
#criterion = nn.MSELoss() # MeanSquaredloss
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent

### Need to figure out how to use mini-batches to gain time in this configuration.
#### train_model is the method used to update the weights.

In [253]:
def train_model_3(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            output = model(train_input.narrow(0, b, mini_batch_size))
            target = train_target.narrow(0, b, mini_batch_size)
            
            print("Time : Output vs target",b,output,target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

       

    

In [233]:
train_model_3(model2,optimizer,nb_epochs,training_data,training_targets,mini_batch_size)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time : Output vs target 0 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281

Time : Output vs target 50 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.028

Time : Output vs target 80 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.028

Time : Output vs target 105 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.02

...

Time : Output vs target 75 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.028

Time : Output vs target 0 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281

Time : Output vs target 25 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.028

Time : Output vs target 55 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.028

Time : Output vs target 80 tensor([[0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.0281, 0.0290, 0.0290, 0.0261, 0.0249,
         0.0243],
        [0.0242, 0.0282, 0.0290, 0.0232, 0.0304, 0.0281, 0.0294, 0.0234, 0.0260,
         0.0232, 0.0300, 0.0240, 0.0287, 0.0301, 0.0271, 0.0274, 0.0264, 0.0287,
         0.0238, 0.0252, 0.0300, 0.0270, 0.0282, 0.0305, 0.0296, 0.0266, 0.0245,
         0.0244, 0.0285, 0.0277, 0.0252, 0.028

In [254]:
model2(training_data[0])

tensor([[0.0000, 0.0000, 0.0000, 0.1001, 0.0000, 0.0000, 0.0932, 0.0000, 0.0000,
         0.0477, 0.0000, 0.0000, 0.0000, 0.1031, 0.0296, 0.0000, 0.0226, 0.0000,
         0.0843, 0.0000, 0.0000, 0.0821, 0.0000, 0.0000, 0.0140, 0.0000, 0.0000,
         0.0099, 0.0000, 0.0000, 0.0000, 0.0000, 0.0812, 0.0391, 0.0270, 0.0409,
         0.0000]], grad_fn=<ReluBackward0>)

In [255]:
model2(training_data[1])

tensor([[0.0000, 0.0000, 0.0000, 0.1001, 0.0000, 0.0000, 0.0932, 0.0000, 0.0000,
         0.0477, 0.0000, 0.0000, 0.0000, 0.1031, 0.0296, 0.0000, 0.0226, 0.0000,
         0.0843, 0.0000, 0.0000, 0.0821, 0.0000, 0.0000, 0.0140, 0.0000, 0.0000,
         0.0099, 0.0000, 0.0000, 0.0000, 0.0000, 0.0812, 0.0391, 0.0270, 0.0409,
         0.0000]], grad_fn=<ReluBackward0>)

In [264]:
training_data[0] == training_data[1]

tensor([[[[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         ...,

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]],

         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]]]], dtype=torch.uint8)

In [262]:
training_data[1]

tensor([[[[ 3.1416,  0.0000,  0.0000,  ...,  3.1416,  3.1416,  3.1416],
          [ 3.1416,  3.1416,  3.1416,  ...,  3.1416,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  3.1416,  ...,  3.1416,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  3.1416,  ...,  0.4645, -0.9849,  0.2861]],

         [[-1.0599, -0.2983,  0.3520,  ...,  0.0212,  1.1990, -2.4110],
          [ 1.5820, -1.7810, -1.9630,  ...,  1.0081, -1.2581, -2.5067],
          [ 0.2541, -1.7595, -2.6920,  ..., -2.2084,  2.3631, -0.9311],
          [ 2.4969,  2.0859, -2.4561,  ...,  0.0218,  0.2800, -0.7354]],

         [[-0.0530, -0.4854,  0.3220,  ..., -0.3965, -0.1348,  0.3742],
          [-1.4514,  1.5488,  2.4034,  ..., -2.7972, -1.7970,  0.4707],
          [-2.1206,  0.7984,  1.2400,  ...,  1.7246,  2.4868,  1.0952],
          [ 1.8198, -1.9030,  1.8566,  ...,  2.6947, -1.7585, -1.0155]],

         ...,

         [[ 0.3559,  1.7527,  3.0559,  ..., -2.4285, -2.3299,  2.9255],
          [-1.0165, -0.4476,  0.5288,  ...,